# Chest-prediction

## data cleaning

In [1]:
import numpy as np
import pandas as pd

import os
from pathlib import Path
import subprocess

In [2]:
USERNAME = os.environ.get('USER')
PROJECT_NAME = "chest-predictor"

In [3]:
LOCAL_DATA_PATH = Path(f"/home/{USERNAME}/code/sachamagier/{PROJECT_NAME}/raw_data/images")

In [4]:
if LOCAL_DATA_PATH.is_dir():
    print("Load local data...")
    # loading data into data

else:
    print("Downloading data from Kaggle...")
    # downloading data from kaggle and saving it into 'raw_data'
    subprocess.run(["kaggle", "datasets", "download", "-d", "nih-chest-xrays/data", "-p", str(LOCAL_DATA_PATH)])


Traceback (most recent call last):
  File "/Users/arnodebelle/.pyenv/versions/chest_predictor/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Users/arnodebelle/.pyenv/versions/3.10.6/envs/chest_predictor/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/Users/arnodebelle/.pyenv/versions/3.10.6/envs/chest_predictor/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /Users/arnodebelle/.kaggle. Or use the environment method.
